In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import Utils.dataframe as dataframe_helper
import Utils.learning as learning


In [3]:
pd.set_option('display.max_columns', None)

## Feature Engineering

In [4]:
def feature_engineering_one_hot(df:pd.DataFrame, N:int = None):
    with open("../Data/top_word/top_fake_word.txt", "r") as f:
        fake_words = dataframe_helper.string_to_list(f.read())
  
    with open("../Data/top_word/top_genuine_word.txt", "r") as f:
        non_fake_words = dataframe_helper.string_to_list(f.read())

    fake_words = fake_words[:N]
    non_fake_words = non_fake_words[:N]
        
    # KL one hot encoding
    fakeWordOneHot = []
    nonFakeWordOneHot = []

    for content in df['reviewContent']:
        fakeOneHot = ''
        nonFakeOneHot = ''
        words = re.findall(r'\b[A-Za-z][a-z]{2,9}\b', content)

        for word in fake_words:
            if word in words:
                fakeOneHot += '1'
            else:
                fakeOneHot += '0'

        for word in non_fake_words:
            if word in words:
                nonFakeOneHot += '1'
            else:
                nonFakeOneHot += '0'
                
        fakeWordOneHot.append(fakeOneHot)
        nonFakeWordOneHot.append(nonFakeOneHot)
    
    result = pd.DataFrame()
    result['fakeWordsOneHot'] = fakeWordOneHot
    result['nonFakeWordsOneHot'] = nonFakeWordOneHot

    result = dataframe_helper.onehot(result, 'fakeWordsOneHot', fake_words, 'fake')
    result = dataframe_helper.onehot(result, 'nonFakeWordsOneHot', non_fake_words, 'non fake')
    result['reviewContent'] = df['reviewContent']
    result['flagged'] = df['flagged']
    
    return result
    

In [5]:
def feature_engineering_word_count(df:pd.DataFrame, N:int = None):
    with open("../Data/top_word/top_fake_word.txt", "r") as f:
        fake_words = dataframe_helper.string_to_list(f.read())
  
    with open("../Data/top_word/top_genuine_word.txt", "r") as f:
        non_fake_words = dataframe_helper.string_to_list(f.read())

    fake_words = fake_words[:N]
    non_fake_words = non_fake_words[:N]
        
    # KL word count
    fakeWordsCount = []
    nonFakeWordsCount = []

    for content in df['reviewContent']:
        words = re.findall(r'\b[A-Za-z][a-z]{2,9}\b', content)
        fakeCount = 0
        nonFakeCount = 0
        for word in words:
            if word in fake_words:
                fakeCount += 1
            elif word in non_fake_words:
                nonFakeCount += 1
        fakeWordsCount.append(fakeCount)
        nonFakeWordsCount.append(nonFakeCount)

    result = pd.DataFrame()
    result['fakeWordsCount'] = fakeWordsCount
    result['nonFakeWordsCount'] = nonFakeWordsCount
    result['reviewContent'] = df['reviewContent']
    result['flagged'] = df['flagged']
    
    return result
    

## Make train dataset

In [6]:
df = dataframe_helper.load_data()
df = dataframe_helper.data_cleaning(df)
df = feature_engineering_one_hot(df)
# df = feature_engineering_word_count(df)

c:\Users\Um_25\Desktop\project\yelp-fake-reviews\Code\..\Utils\dataframe.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i] = df['date'][i][1:]
c:\Users\Um_25\Desktop\project\yelp-fake-reviews\Code\..\Utils\dataframe.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col_name[i]} ({type})'] = [data[i] for data in df[col]]


In [7]:
df.head()

,food (fake),great (fake),service (fake),place (fake),restaurant (fake),best (fake),staff (fake),owner (fake),time (fake),pizza (fake),love (fake),manager (fake),always (fake),never (fake),would (fake),atmosphere (fake),told (fake),simply (fake),sushi (fake),asked (fake),went (fake),customer (fake),recommend (fake),waiter (fake),friends (fake),chicago (fake),family (fake),rude (fake),better (fake),wonderful (fake),eat (fake),going (fake),waitress (fake),minutes (fake),worst (fake),people (fake),vietnamese (fake),bad (fake),money (fake),customers (fake),experience (fake),prices (fake),friendly (fake),times (fake),table (fake),back (fake),years (fake),server (fake),wife (fake),italian (fake),fresh (fake),business (fake),birthday (fake),bill (fake),terrible (fake),party (fake),husband (fake),busy (fake),delivery (fake),ever (fake),city (fake),said (fake),bar (fake),week (fake),excellent (fake),amazing (fake),mexican (fake),last (fake),took (fake),new (fake),many (fake),price (fake),pho (fake),quality (fake),horrible (fake),say (fake),hostess (fake),called (fake),fogo (fake),far (fake),first (fake),highly (fake),sunda (fake),authentic (fake),appetizers (fake),favorite (fake),coast (fake),extremely (fake),steak (fake),nana (fake),treated (fake),left (fake),away (fake),evening (fake),make (fake),nice (fake),absolutely (fake),everyone (fake),care (fake),places (fake),persian (fake),town (fake),give (fake),offered (fake),sat (fake),kids (fake),byob (fake),live (fake),anyone (fake),fantastic (fake),work (fake),walked (fake),pay (fake),gave (fake),one (fake),attitude (fake),paid (fake),brazilian (fake),spacca (fake),girlfriend (fake),night (fake),lincoln (fake),management (fake),several (fake),host (fake),hands (fake),nuevo (fake),past (fake),guests (fake),reasonable (fake),soon (fake),brazzaz (fake),wait (fake),korean (fake),tuan (fake),leon (fake),poor (fake),take (fake),visit (fake),call (fake),owners (fake),received (fake),order (fake),every (fake),special (fake),attentive (fake),needs (fake),suggest (fake),average (fake),another (fake),waiting (fake),leave (fake),beef (fake),die (fake),phone (fake),seated (fake),charge (fake),park (fake),wanted (fake),music (fake),must (fake),fabulous (fake),charged (fake),looking (fake),napoli (fake),thanks (fake),tried (fake),even (fake),steaks (fake),waste (fake),card (fake),young (fake),check (fake),entrees (fake),ago (fake),overpriced (fake),job (fake),worse (fake),awful (fake),late (fake),enjoy (fake),guy (fake),visiting (fake),tip (fake),extra (fake),indian (fake),pricey (fake),event (fake),greeted (fake),sent (fake),daughter (fake),weekend (fake),employees (fake),dine (fake),dollars (fake),italy (fake),delivered (fake),brought (fake),main (fake),expensive (fake),ask (fake),understand (fake),loud (fake),apology (fake),person (fake),chris (fake),kabab (fake),managers (fake),making (fake),waited (fake),gets (fake),rolls (fake),credit (fake),incredible (fake),name (fake),speak (fake),dirty (fake),cant (fake),class (fake),sprout (fake),tasteless (fake),total (fake),attention (fake),villa (fake),cake (fake),mastro (fake),real (fake),appetizer (fake),showed (fake),gino (fake),return (fake),today (fake),response (fake),asian (fake),telling (fake),welcoming (fake),jones (fake),frontera (fake),taking (fake),agree (fake),mediocre (fake),pizzas (fake),year (fake),mention (fake),rudely (fake),thats (fake),priced (fake),trying (fake),chao (fake),policy (fake),tell (fake),free (fake),min (fake),cannot (fake),opened (fake),saying (fake),waiters (fake),support (fake),talk (fake),margaritas (fake),organic (fake),needed (fake),waitresses (fake),lemongrass (fake),refused (fake),finally (fake),ready (fake),garcia (fake),paying (fake),bring (fake),lady (fake),correct (fake),change (fake),twice (fake),pleasant (fake),ruth (fake),calo (fake),often (fake),treat (fake),dont (fake),downtown (fake),worth (fake),keep (fake),american (fake),everytime (fake),desert (fake),thank (fake),someone (fake),wontons (fake),taken 

In [8]:
under_sampled_df = dataframe_helper.under_sampling(df=df, target='flagged', big_sample='Y', small_sample='N')
under_sampled_df.reset_index(drop=True, inplace=True)
# under_sampled_df.to_csv('../Data/under_sampled_KL_df.csv', index=False)
# under_sampled_df = pd.read_csv('../Data/under_sampled_KL_df.csv')

Under-Sampling Data
Under-Sampling Complete


In [9]:
over_sampled_df = dataframe_helper.over_sampling(df=df, target='flagged', big_sample='Y', small_sample='N')
over_sampled_df.reset_index(drop=True, inplace=True)
# over_sampled_df.to_csv('../Data/over_sampled_KL_df.csv', index=False)
# over_sampled_df = pd.read_csv('../Data/over_sampled_KL_df.csv')

Over-Sampling Data
Over-Sampling Complete


## Model

In [10]:
rf = RandomForestClassifier(random_state=42, criterion='entropy', max_depth=14, max_features='auto',
                            n_estimators=500)
nb = GaussianNB()

In [11]:
# model, results, feature = learning.semi_supervised_learning(df, model=rf, threshold=0.7, iterations=15, algorithm='Random Forest')
# model, results, feature = learning.semi_supervised_learning(df, model=nb, threshold=0.7, iterations=15, algorithm='Naive Bayes')

In [12]:
# model, results, feature = learning.semi_supervised_learning(under_sampled_df, model=rf, threshold=0.7, iterations=15, algorithm='Random Forest')
# model, results, feature = learning.semi_supervised_learning(under_sampled_df, model=nb, threshold=0.7, iterations=15, algorithm='Naive Bayes')

In [13]:
model, results, feature = learning.semi_supervised_learning(over_sampled_df, model=rf, threshold=0.7, iterations=15, algorithm='Random Forest', target_column='flagged')
# model, results, feature = learning.semi_supervised_learning(over_sampled_df, model=nb, threshold=0.7, iterations=15, algorithm='Naive Bayes')

Training Random Forest Model


  0%|          | 0/15 [00:00<?, ?it/s]

ValueError: could not convert string to float: 'N'

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=feature)
feat_importances.nlargest(100).plot(kind='barh',figsize=(25,25))

In [ ]:
# sort = model.feature_importances_.argsort()
# plt.barh(boston.feature_names[sort], model.feature_importances_[sort])
# plt.xlabel("Feature Importance")